In [5]:
import requests
import json

# Set the base URL (assumes Ollama is running on localhost:11434)
OLLAMA_URL = 'http://localhost:11434'

def get_available_models():
    """Get list of available models from Ollama"""
    try:
        response = requests.get(f'{OLLAMA_URL}/api/tags')
        if response.ok:
            models = response.json().get('models', [])
            return [model['name'] for model in models]
        return []
    except Exception as e:
        print(f"Error getting available models: {e}")
        return []

def chat_with_model(model_name, user_input):
    """
    Chat with specified model (llama or deepseek)
    
    Args:
        model_name (str): Name of the model to use ('llama' or 'deepseek')
        user_input (str): The user's input message
    """
    # Validate model name
    available_models = get_available_models()
    if not available_models:
        print("Could not fetch available models. Please ensure Ollama is running.")
        return
    
    # Map model names to their full Ollama model names
    model_mapping = {
        'llama': 'llama3:latest',  # or 'llama3' depending on what you have installed
        'deepseek': 'deepseek-coder:latest'
    }
    
    full_model_name = model_mapping.get(model_name.lower())
    if not full_model_name:
        print(f"Invalid model name. Please choose from: {', '.join(model_mapping.keys())}")
        return
    
    if full_model_name not in available_models:
        print(f"Model {full_model_name} is not available. Available models are: {', '.join(available_models)}")
        return

    response = requests.post(
        f'{OLLAMA_URL}/api/chat',
        json={
            'model': full_model_name,
            'messages': [
                {'role': 'user', 'content': user_input}
            ],
            'stream': False
        }
    )

    if response.ok:
        try:
            reply = response.json()['message']['content']
            print(f"{full_model_name} says:", reply)
        except (KeyError, json.JSONDecodeError) as e:
            print("Error parsing response:", e)
            print("Raw response:", response.text)
    else:
        print("Error:", response.text)

# Example usage
print("Available models:", get_available_models())
print("\nExample with llama:")
chat_with_model('llama', "Explain quantum physics in simple terms.")
print("\nExample with deepseek:")
chat_with_model('deepseek', "Write a Python function to calculate fibonacci numbers.")

Available models: ['deepseek-coder:latest', 'llama3:latest']

Example with llama:
llama3:latest says: Quantum physics! It's a mind-blowing field that explores the tiny, hidden world of particles and forces. I'll try to simplify it for you:

**The Basics:**

Quantum physics is a branch of physics that studies the behavior of extremely small things like atoms, electrons, and photons (tiny particles of light). These tiny particles don't follow the same rules as bigger objects do. They're weird!

**Key Concepts:**

1. **Wave-Particle Duality:** Tiny particles can behave like both waves and particles! Imagine a wave on the ocean; it has peaks and troughs. Now, imagine a particle that can be in multiple places at once, like a wave overlapping itself.
2. **Uncertainty Principle:** You can't know everything about a tiny particle at the same time. The more you try to measure its position, speed, or energy, the less you can know others. It's like trying to pin down a slippery fish!
3. **Superpos